In [ ]:
import vispy
from vispy import scene
from vispy.scene import visuals
import numpy as np

# Generate sample graph data
nodes = np.array([
    [0, 0],
    [1, 0],
    [0.5, 0.866],
    [0.5, -0.866]
])

edges = np.array([
    [0, 1],
    [0, 2],
    [0, 3],
    [1, 2],
    [1, 3],
])

# Assign random values to edges
edge_values = np.random.rand(len(edges))

# Create the canvas with a white background and add a view
canvas = scene.SceneCanvas(keys='interactive', show=True, bgcolor='white')
view = canvas.central_widget.add_view()

# Create scatter plot for nodes
scatter = visuals.Markers()
scatter.set_data(nodes, face_color='blue', size=10)
view.add(scatter)

# Create lines for edges
lines = visuals.Line(pos=nodes[edges].reshape(-1, 2),
                     connect='segments',
                     color='black',  # Change edge color to black for contrast
                     width=2)
view.add(lines)

# Set the camera to 2D
view.camera = 'panzoom'
view.camera.aspect = 1

# Create a Text Visual for displaying information
text = visuals.Text('', color='black', pos=(10, 10), font_size=12, parent=canvas.scene)

# Function to handle mouse movement and display info
def on_mouse_move(event):
    if event.is_dragging:
        return

    if event.pos is None:
        return

    # Map the mouse position to the scene
    tr = view.scene.transform
    mouse_pos = tr.imap(event.pos)[:2]

    # Check if mouse is over a node
    distances = np.linalg.norm(nodes - mouse_pos, axis=1)
    min_dist = np.min(distances)
    if min_dist < 0.05:  # Threshold for detecting node hover
        index = np.argmin(distances)
        node_coord = nodes[index]
        text.text = f'Node {index}: ({node_coord[0]:.2f}, {node_coord[1]:.2f})'
        text.pos = event.pos
        return

    # Check if mouse is over an edge
    min_edge_dist = None
    min_edge_index = None
    for i, (start_idx, end_idx) in enumerate(edges):
        start_point = nodes[start_idx]
        end_point = nodes[end_idx]

        # Compute projection of mouse position onto the edge
        edge_vec = end_point - start_point
        point_vec = mouse_pos - start_point
        edge_length = np.linalg.norm(edge_vec)
        if edge_length == 0:
            continue
        t = np.dot(point_vec, edge_vec) / (edge_length ** 2)
        t = np.clip(t, 0, 1)
        projection = start_point + t * edge_vec
        distance = np.linalg.norm(mouse_pos - projection)

        # Update minimum distance and edge index
        if min_edge_dist is None or distance < min_edge_dist:
            min_edge_dist = distance
            min_edge_index = i

    if min_edge_dist is not None and min_edge_dist < 0.05:  # Threshold for edge hover
        edge = edges[min_edge_index]
        value = edge_values[min_edge_index]
        text.text = f'Edge {edge.tolist()}: Value {value:.2f}'
        text.pos = event.pos
    else:
        text.text = ''

# Connect the mouse move event
canvas.events.mouse_move.connect(on_mouse_move)

canvas